In [17]:
import os
os.environ["PYSPARK_PYTHON"] = "python3.10"
os.environ["PYSPARK_DRIVER_PYTHON"] = "python3.10"

In [18]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").config("spark.driver.memory", "32g").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) #  This will format our output tables a bit nicer when not using the show() method
spark

In [19]:
sc = spark.sparkContext

In [20]:
books = sc.textFile("datasets/merged_books_ratings.csv")
genres = sc.textFile("datasets/Genres.csv")
ratings = sc.textFile("datasets/Ratings.csv")

In [21]:
books.take(1)

['user_id,isbn,rating,authors,description,genres,pages,title,publishYear,publishMonth,publishDay']

In [22]:
genres.take(1)

['isbn,genre']

In [23]:
import time

def isbn_books(row):
    fields = row.split(",")
    return (fields[1], fields[2])

def isbn_genres(row):
    fields = row.split(",")
    return (fields[0], fields[1])

books_part = books.map(isbn_books)
genres_part = genres.map(isbn_genres)

books_part = books_part.partitionBy(8)
genres_part = genres_part.partitionBy(8)

join_rdd = books_part.join(genres_part) \
                .map(lambda pairs: pairs[1])
start = time.perf_counter()
join_output = join_rdd.collect()
end = time.perf_counter()

In [8]:
print("Execution time: " + str(end - start))

Execution time: 22.767871525999908


In [13]:

ratings_rdd = ratings.map(lambda row: row.split(",")) \
                     .map(lambda row: (row[1], row[2])) \
                     .reduceByKey(lambda x, y: int(x)+int(y))
start = time.perf_counter()
ratings_output = ratings_rdd.collect()
end = time.perf_counter()

In [14]:
print("Execution time: " + str(end - start))

Execution time: 16.39711255200018


In [15]:
print("Execution time: " + str(end - start))

Execution time: 16.39711255200018


In [16]:
spark.stop()